In [1]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

In [4]:
# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [5]:
# Image processing
# transform = transforms.Compose([
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
#                                      std=(0.5, 0.5, 0.5))])
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5],   # 1 for greyscale channels
                                     std=[0.5])])

In [6]:
# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)


In [7]:
# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [8]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

In [9]:
# Device setting
D = D.to(device)
G = G.to(device)

In [10]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [11]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [12]:
# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))


Epoch [0/200], Step [200/600], d_loss: 0.0292, g_loss: 4.3150, D(x): 1.00, D(G(z)): 0.03
Epoch [0/200], Step [400/600], d_loss: 0.0882, g_loss: 5.5306, D(x): 0.98, D(G(z)): 0.06
Epoch [0/200], Step [600/600], d_loss: 0.0171, g_loss: 6.1078, D(x): 0.99, D(G(z)): 0.01
Epoch [1/200], Step [200/600], d_loss: 0.1269, g_loss: 4.9534, D(x): 0.93, D(G(z)): 0.04
Epoch [1/200], Step [400/600], d_loss: 0.4243, g_loss: 2.5779, D(x): 0.86, D(G(z)): 0.17
Epoch [1/200], Step [600/600], d_loss: 0.1068, g_loss: 5.0011, D(x): 0.96, D(G(z)): 0.05
Epoch [2/200], Step [200/600], d_loss: 0.0770, g_loss: 4.6138, D(x): 0.96, D(G(z)): 0.03
Epoch [2/200], Step [400/600], d_loss: 0.1669, g_loss: 3.4041, D(x): 0.95, D(G(z)): 0.07
Epoch [2/200], Step [600/600], d_loss: 0.1732, g_loss: 3.7711, D(x): 0.93, D(G(z)): 0.07
Epoch [3/200], Step [200/600], d_loss: 0.2151, g_loss: 3.1231, D(x): 0.95, D(G(z)): 0.13
Epoch [3/200], Step [400/600], d_loss: 1.1233, g_loss: 1.4855, D(x): 0.72, D(G(z)): 0.39
Epoch [3/200], Step [

Epoch [30/200], Step [600/600], d_loss: 0.2929, g_loss: 2.8415, D(x): 0.89, D(G(z)): 0.08
Epoch [31/200], Step [200/600], d_loss: 0.2915, g_loss: 3.2970, D(x): 0.92, D(G(z)): 0.11
Epoch [31/200], Step [400/600], d_loss: 0.2927, g_loss: 3.9216, D(x): 0.93, D(G(z)): 0.13
Epoch [31/200], Step [600/600], d_loss: 0.2061, g_loss: 5.5003, D(x): 0.93, D(G(z)): 0.07
Epoch [32/200], Step [200/600], d_loss: 0.5131, g_loss: 3.7381, D(x): 0.79, D(G(z)): 0.06
Epoch [32/200], Step [400/600], d_loss: 0.3816, g_loss: 4.4639, D(x): 0.87, D(G(z)): 0.07
Epoch [32/200], Step [600/600], d_loss: 0.3933, g_loss: 3.5509, D(x): 0.87, D(G(z)): 0.08
Epoch [33/200], Step [200/600], d_loss: 0.1671, g_loss: 4.3783, D(x): 0.95, D(G(z)): 0.09
Epoch [33/200], Step [400/600], d_loss: 0.2836, g_loss: 2.4731, D(x): 0.91, D(G(z)): 0.11
Epoch [33/200], Step [600/600], d_loss: 0.4725, g_loss: 3.2809, D(x): 0.82, D(G(z)): 0.07
Epoch [34/200], Step [200/600], d_loss: 0.4952, g_loss: 2.6937, D(x): 0.86, D(G(z)): 0.15
Epoch [34/

Epoch [61/200], Step [400/600], d_loss: 0.8916, g_loss: 1.7245, D(x): 0.84, D(G(z)): 0.37
Epoch [61/200], Step [600/600], d_loss: 0.6426, g_loss: 2.0642, D(x): 0.76, D(G(z)): 0.16
Epoch [62/200], Step [200/600], d_loss: 0.8450, g_loss: 2.1471, D(x): 0.69, D(G(z)): 0.19
Epoch [62/200], Step [400/600], d_loss: 0.5514, g_loss: 1.9602, D(x): 0.83, D(G(z)): 0.20
Epoch [62/200], Step [600/600], d_loss: 0.4610, g_loss: 2.5053, D(x): 0.83, D(G(z)): 0.17
Epoch [63/200], Step [200/600], d_loss: 0.5401, g_loss: 2.0401, D(x): 0.83, D(G(z)): 0.18
Epoch [63/200], Step [400/600], d_loss: 0.4331, g_loss: 3.0014, D(x): 0.81, D(G(z)): 0.13
Epoch [63/200], Step [600/600], d_loss: 0.5209, g_loss: 2.3434, D(x): 0.85, D(G(z)): 0.20
Epoch [64/200], Step [200/600], d_loss: 0.6639, g_loss: 2.4513, D(x): 0.85, D(G(z)): 0.28
Epoch [64/200], Step [400/600], d_loss: 0.7163, g_loss: 2.5636, D(x): 0.84, D(G(z)): 0.30
Epoch [64/200], Step [600/600], d_loss: 0.6815, g_loss: 2.2638, D(x): 0.80, D(G(z)): 0.24
Epoch [65/

Epoch [92/200], Step [200/600], d_loss: 0.9779, g_loss: 2.3226, D(x): 0.62, D(G(z)): 0.17
Epoch [92/200], Step [400/600], d_loss: 1.0201, g_loss: 1.7882, D(x): 0.57, D(G(z)): 0.18
Epoch [92/200], Step [600/600], d_loss: 0.8868, g_loss: 1.8178, D(x): 0.78, D(G(z)): 0.34
Epoch [93/200], Step [200/600], d_loss: 0.7663, g_loss: 1.7709, D(x): 0.82, D(G(z)): 0.31
Epoch [93/200], Step [400/600], d_loss: 0.7834, g_loss: 2.0735, D(x): 0.72, D(G(z)): 0.23
Epoch [93/200], Step [600/600], d_loss: 0.8125, g_loss: 1.5058, D(x): 0.77, D(G(z)): 0.30
Epoch [94/200], Step [200/600], d_loss: 0.8080, g_loss: 2.4817, D(x): 0.68, D(G(z)): 0.19
Epoch [94/200], Step [400/600], d_loss: 0.9879, g_loss: 1.6278, D(x): 0.72, D(G(z)): 0.34
Epoch [94/200], Step [600/600], d_loss: 0.7219, g_loss: 2.1335, D(x): 0.71, D(G(z)): 0.16
Epoch [95/200], Step [200/600], d_loss: 0.6933, g_loss: 2.6911, D(x): 0.77, D(G(z)): 0.23
Epoch [95/200], Step [400/600], d_loss: 0.9560, g_loss: 1.9022, D(x): 0.75, D(G(z)): 0.33
Epoch [95/

Epoch [122/200], Step [400/600], d_loss: 0.7805, g_loss: 1.6077, D(x): 0.78, D(G(z)): 0.32
Epoch [122/200], Step [600/600], d_loss: 0.7314, g_loss: 1.9095, D(x): 0.77, D(G(z)): 0.28
Epoch [123/200], Step [200/600], d_loss: 0.8608, g_loss: 2.2754, D(x): 0.71, D(G(z)): 0.25
Epoch [123/200], Step [400/600], d_loss: 0.9837, g_loss: 1.4019, D(x): 0.66, D(G(z)): 0.26
Epoch [123/200], Step [600/600], d_loss: 0.7186, g_loss: 2.0225, D(x): 0.75, D(G(z)): 0.24
Epoch [124/200], Step [200/600], d_loss: 0.7556, g_loss: 1.9339, D(x): 0.68, D(G(z)): 0.20
Epoch [124/200], Step [400/600], d_loss: 0.8085, g_loss: 2.1288, D(x): 0.77, D(G(z)): 0.32
Epoch [124/200], Step [600/600], d_loss: 0.6956, g_loss: 1.5581, D(x): 0.74, D(G(z)): 0.22
Epoch [125/200], Step [200/600], d_loss: 0.8714, g_loss: 1.6964, D(x): 0.66, D(G(z)): 0.21
Epoch [125/200], Step [400/600], d_loss: 0.8517, g_loss: 1.5358, D(x): 0.77, D(G(z)): 0.35
Epoch [125/200], Step [600/600], d_loss: 0.8804, g_loss: 1.9501, D(x): 0.67, D(G(z)): 0.26

Epoch [152/200], Step [600/600], d_loss: 1.0364, g_loss: 1.3534, D(x): 0.72, D(G(z)): 0.38
Epoch [153/200], Step [200/600], d_loss: 0.8498, g_loss: 1.6110, D(x): 0.68, D(G(z)): 0.22
Epoch [153/200], Step [400/600], d_loss: 0.9892, g_loss: 1.6191, D(x): 0.67, D(G(z)): 0.31
Epoch [153/200], Step [600/600], d_loss: 1.0620, g_loss: 1.5404, D(x): 0.63, D(G(z)): 0.32
Epoch [154/200], Step [200/600], d_loss: 1.1483, g_loss: 0.9823, D(x): 0.63, D(G(z)): 0.35
Epoch [154/200], Step [400/600], d_loss: 1.0276, g_loss: 1.2386, D(x): 0.78, D(G(z)): 0.43
Epoch [154/200], Step [600/600], d_loss: 0.9247, g_loss: 1.4282, D(x): 0.75, D(G(z)): 0.37
Epoch [155/200], Step [200/600], d_loss: 0.8196, g_loss: 1.6366, D(x): 0.73, D(G(z)): 0.27
Epoch [155/200], Step [400/600], d_loss: 0.8712, g_loss: 1.9868, D(x): 0.67, D(G(z)): 0.22
Epoch [155/200], Step [600/600], d_loss: 1.0390, g_loss: 1.3231, D(x): 0.66, D(G(z)): 0.34
Epoch [156/200], Step [200/600], d_loss: 0.8904, g_loss: 1.8600, D(x): 0.68, D(G(z)): 0.26

Epoch [183/200], Step [200/600], d_loss: 0.8245, g_loss: 1.4145, D(x): 0.75, D(G(z)): 0.34
Epoch [183/200], Step [400/600], d_loss: 0.9401, g_loss: 1.2299, D(x): 0.68, D(G(z)): 0.33
Epoch [183/200], Step [600/600], d_loss: 1.1295, g_loss: 1.2988, D(x): 0.58, D(G(z)): 0.29
Epoch [184/200], Step [200/600], d_loss: 0.9618, g_loss: 1.2933, D(x): 0.71, D(G(z)): 0.36
Epoch [184/200], Step [400/600], d_loss: 0.9575, g_loss: 1.3151, D(x): 0.68, D(G(z)): 0.32
Epoch [184/200], Step [600/600], d_loss: 1.0260, g_loss: 1.3907, D(x): 0.64, D(G(z)): 0.29
Epoch [185/200], Step [200/600], d_loss: 0.9006, g_loss: 1.3405, D(x): 0.72, D(G(z)): 0.31
Epoch [185/200], Step [400/600], d_loss: 0.7945, g_loss: 1.7822, D(x): 0.69, D(G(z)): 0.24
Epoch [185/200], Step [600/600], d_loss: 1.0270, g_loss: 1.9155, D(x): 0.66, D(G(z)): 0.28
Epoch [186/200], Step [200/600], d_loss: 1.0727, g_loss: 1.5352, D(x): 0.65, D(G(z)): 0.33
Epoch [186/200], Step [400/600], d_loss: 0.8329, g_loss: 1.7362, D(x): 0.73, D(G(z)): 0.32

In [13]:
# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')